In [1]:
# 分析対象銘柄の証券コードをセット
code = 1301

In [2]:
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

In [3]:
from lib_dataprocess import KessanPl, read_data

In [9]:
import polars as pl

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

df1 = KPL.get_latest_yearly_settlements(settlement_type="予")
df2 = KPL.get_latest_yearly_settlements(settlement_type="本")

df = pl.concat([df1, df2])
df.filter(pl.col("code")==code)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
1301,2025-03-31,"""予""",2024-05-10,300000,10000,10000,7000,589.4,110.0,4
1301,2024-03-31,"""本""",2024-05-10,261604,8806,8856,5936,548.6,100.0,4
